# Homework 1
Megan McGee
May 17, 2022

In [1]:
# imports
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df_jan = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
df_feb = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

In [3]:
# question #1: how many records in january dataset: 1154112
df_jan.shape

(1154112, 7)

In [4]:
df_jan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [5]:
# create column of duration in minutes and calculate average duration
df_jan['duration'] = df_jan.dropOff_datetime - df_jan.pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

df_feb['duration'] = df_feb.dropOff_datetime - df_feb.pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
# question #2: average duration of a trip for january: 19.16
df_jan.duration.mean()

19.1672240937939

In [7]:
df_jan.duration.describe()

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
25%      7.766667e+00
50%      1.340000e+01
75%      2.228333e+01
max      4.233710e+05
Name: duration, dtype: float64

In [8]:
# keep only the records where duration is between 1 and 60 minutes inclusive
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [9]:
df_jan.shape

(1109826, 8)

In [10]:
1154112 - 1109826

44286

### Dropping records
- 1154112 records initially
- 1109826 records after dropping outliers
- 44286 records dropped

In [11]:
# question #3: fraction of missing data for pickup location ID: 83%
df_jan.PUlocationID.isna().sum() / len(df_jan.PUlocationID)

0.8352732770722617

In [12]:
# fill missing values in PUlocationID and DOlocationID with -1
df_jan['PUlocationID'] = df_jan['PUlocationID'].fillna(-1)
df_jan['DOlocationID'] = df_jan['DOlocationID'].fillna(-1)
df_feb['PUlocationID'] = df_feb['PUlocationID'].fillna(-1)
df_feb['DOlocationID'] = df_feb['DOlocationID'].fillna(-1)

In [13]:
categorical = ['PUlocationID', 'DOlocationID']
df_jan[categorical] = df_jan[categorical].astype(str)
df_feb[categorical] = df_feb[categorical].astype(str)

In [14]:
train_dicts = df_jan[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [15]:
# question #4: number of columns in the feature matrix: 525
X_train.shape

(1109826, 525)

In [16]:
target = 'duration'
y_train = df_jan[target].values
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
# question #5: rmse of the linear regression on the training data: 10.52
y_pred = lr.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
rmse

10.528519107206316

In [18]:
val_dicts = df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_feb[target].values

In [19]:
# question #6: rmse of the linear regression on the validation data: 11.01
y_pred_val = lr.predict(X_val)
rmse = mean_squared_error(y_val, y_pred_val, squared=False)
rmse

11.014283149347039